Interactive version of the tutorial: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/pgrigorev/ModMatEcole/HEAD?labpath=tutorials%2FT2_2_structural_analysis_neighbors_maps.ipynb)

**Setup necessary packages (in binder environment it is already installed)**

In [ ]:
! pip install git+https://github.com/arn-all/neighbors-maps ase plotly scikit-learn matplotlib matscipy numpy tqdm

In [ ]:
import ase
import ase.lattice
import ase.lattice.tetragonal
import ase.lattice.monoclinic
import ase.lattice.triclinic
import ase.lattice.cubic
import ase.lattice.hexagonal
import ase.lattice.orthorhombic

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import numpy as np
from tqdm import tqdm


# For descriptors
from neighbors_map import Atoms

## 1. Create synthetic data

We will use ASE to generate a dataset of simple crystal structures

In [ ]:
# List of 7 Bravais Lattices to consider
# Each entry is a tuple: (crystal system, lattice type, lattice constants)
bravais_lattices = [
    ("cubic", "SimpleCubic", 3.0),  # Simple cubic lattice (single constant)
    ("cubic", "FaceCenteredCubic", 3.0),  # Face-centered cubic (single constant)
    ("cubic", "BodyCenteredCubic", 3.0),  # Body-centered cubic (single constant)
    ("cubic", "Diamond", 3.0),  # Diamond cubic (single constant)
    # For orthorhombic, we need 3 constants: a, b/a (ratio), c/a (ratio)
    ("orthorhombic", "BaseCenteredOrthorhombic", {'a': 3.0, 'b/a': 1.2, 'c/a': 1.5}), 
    ("orthorhombic", "FaceCenteredOrthorhombic", {'a': 3.0, 'b/a': 1.2, 'c/a': 1.7}),
    ("orthorhombic", "BodyCenteredOrthorhombic", {'a': 3.0, 'b/a': 1.2, 'c/a': 1.5}),
]

# Function to Create a Lattice
def generate_lattice(lattice_module, lattice_class, element, size, lattice_constants):
    """
    Creates an atomic lattice using the ASE (Atomic Simulation Environment) library.

    Args:
        lattice_module (str): The crystal system (e.g., "cubic", "orthorhombic")
        lattice_class (str): The specific Bravais lattice type (e.g., "SimpleCubic")
        element (str): The chemical symbol of the element (e.g., "Fe" for iron). This won't change anything for this tutorial.
        size (tuple): The size of the lattice in terms of unit cells (e.g., (4, 4, 4))
        lattice_constants (float or dict): 
            - If float: The lattice constant for cubic systems.
            - If dict: A dictionary of lattice constants for orthorhombic systems.
    
    Returns:
        Atoms: An Atoms object representing the created lattice.
    """
    lattice_type = getattr(getattr(ase.lattice, lattice_module), lattice_class)
    lattice = lattice_type(element, size=size, latticeconstant=lattice_constants)
    return Atoms(lattice)


# Generate Lattices and Store Them as Lists
lattices = []     # List to store the created lattice objects
lattice_num = []  # List of class labels in the form of numbers (will be useful for classification) 
lattice_name = [] # List to store the actual human-readable lattice names 

for i, (lattice_type, lattice_class, lattice_constants) in enumerate(bravais_lattices):
    # 'enumerate' gives us both the index (i) and the tuple from the list
    
    lattices.append(generate_lattice(lattice_type, lattice_class, "Fe", (6, 6, 6), lattice_constants))
    lattice_num.append(i)         
    lattice_name.append(lattice_class)

Here is what we created:

In [ ]:
lattices

In [ ]:
lattice_num

In [ ]:
lattice_name

## 2. On the choice of a cut-off radius 

- The cut-off radius is a critical parameter as it controls the locality assumption that is made when we talk about a *local* atomic environment
- Some processes or transformations will be much easier to detect with a longer cut-off radius, while increased computational cost and signal averaging are more likely for long cut-offs 

In [ ]:
def get_cutoff(atoms, n_neighbors, cutoff=10.):
    """
    Function to get the distance of n_th neighbour using matscipy neighbor list.
    """
    from matscipy.neighbours import neighbour_list  # Import the neighbor list tool

    # Create the neighbor list
    # This finds all pairs of atoms within the 'cutoff' distance and stores their
    # indices (i) and distances (d) in arrays.
    i, d = neighbour_list("id", atoms, cutoff=cutoff)  

    # Focus on the first atom (index 0)
    # This extracts only the distances from the first atom to its neighbors.
    first_atom_d = d[i == 0] 

    # Sort the distances
    first_atom_d.sort()

    # Get the n-th neighbor distance
    # After sorting, the distance at index (n_neighbors) will be the distance to 
    # the n-th nearest neighbor of the first atom.
    return first_atom_d[n_neighbors] 

In [ ]:
# Define the number of neighbors we're interested in
n_neighbors = [24, 32, 64, 128]

# Create a range of 20 cutoff distances to test (from 1 Angstrom to 150 Angstroms)
X = np.linspace(1, 128, 25, dtype=int)  

print(lattice_name[1])

# Calculate cutoff radii for each desired number of neighbors in this structure
cutoff_radii = [get_cutoff(lattices[1], x) for x in X]

# Plot the relationship between cutoff radius and number of neighbors (black line)
plt.plot(cutoff_radii, X, ".-", color='k', alpha=0.8)  


# Add horizontal dashed lines for the desired number of neighbors
for neighbor_count in n_neighbors:
    plt.hlines(neighbor_count, xmin=min(cutoff_radii), xmax=max(cutoff_radii), linestyle=":", alpha=0.5)

# Label the axes
plt.xlabel(r"Cut-off radius $\AA$")  # $\AA$ is the Angstrom symbol in LaTeX
plt.ylabel("Number of neighbors")

# Show the plot
plt.show()


NB: In perfect crystals, the number of atoms within a cut-off radius is not a smooth or linear function.
Above, we show the number of neighbors found within a certain cut-off radius, for a perfect crystal (black)

## 3. Effect of the cut-off radius on atomic descriptors

In [ ]:
def plot_nm(ax, number_of_neighbors_in_rcut=32):
    """
    Plots neighbor maps for each lattice, showing the effect of the specified cutoff radius (rows).

    Args:
        ax: The subplot axes where the neighbor map will be plotted.
        number_of_neighbors_in_rcut (int): The desired number of neighbors within 
            the cutoff radius. Defaults to 32.
    """

    for i, (lattice, bravais) in enumerate(zip(lattices, bravais_lattices)): 

        # Create an Atoms object from the lattice data
        lat = Atoms(lattice)  

        # --- Optional: Adjust Neighbor Map Parameters ---
        # (Uncomment to experiment with different settings)
        # lat.neighbor_map.beta = 3    
        # lat.neighbor_map.gamma = 2 

        # Get the neighbor map using the specified cutoff radius
        cutoff_radius = get_cutoff(lat, number_of_neighbors_in_rcut)
        image = lat.get_neighbor_map(0, r_cut=cutoff_radius)  # 0 means focus on the first atom

        # Display the neighbor map as an image
        ax[i].imshow(image)  
        ax[i].axis("off")   # Turn off axis labels for a cleaner look


# Create a grid of subplots
fig, axs = plt.subplots(len(n_neighbors), len(lattices), figsize=(6, 5), dpi=300, constrained_layout=True)

# Generate neighbor maps for different numbers of neighbors (rows)
for i, n in enumerate(n_neighbors): 
    plot_nm(ax=axs[i], number_of_neighbors_in_rcut=n)

# Add titles above each column (lattice type)
for i, _ in enumerate(lattices):
    for j, n_neigh in enumerate(n_neighbors):
        axs[j, i].set_title(f"{bravais_lattices[i][1]}\n n={n_neigh}", fontsize=4)  


plt.suptitle(r"Neighbors maps with an increasingly large number of atoms in $r_{cut}$", size=6)
plt.show()

- Visually, do the images above (for a given row) seem different enough to be classified ?
- **Images stongly depend on the cut-off radius -> comparison should be done for a given rcut and values of beta and gamma**

- These images are for perfect crystalline structures. A descriptor is useful only if it can recognize them in the wild (i.e. in the presence of noise)

## 4. Effect of uncorrelated noise on neighbors maps 

- How is the structure of neighbors maps affected by noise ?

**Remark** Adding physical, correlated thermal noise would require a MD engine and an interaction model suited for each crystal structures.

Here, for simplicity, we use uncorrelated, Gaussian distributed, noise on the atoms. Note that this is not physical and will eventually lead to atoms overlapping for large displacements (atoms are not repelled by each other).

In [ ]:
def noisy_structure(atoms, noise):
    """
    Creates a new atomic structure with added noise to the atom positions.

    Args:
        atoms: The original Atoms object representing the atomic structure.
        noise: A float representing the standard deviation (spread) of the 
               Gaussian noise to be added to each atom's position in each direction (x, y, z).

    Returns:
        Atoms: A new Atoms object with the perturbed (noisy) atomic positions.
    """

    noisy_atoms = atoms.copy()
    noise = np.random.normal(loc=0, scale=noise, size=(atoms.positions.shape[0], 3))
    noisy_atoms.positions += noise
    return Atoms(noisy_atoms)

In [ ]:
number_of_neighbors_in_rcut = 64
noise_levels = [0, 0.01, 0.05, 0.1, 0.15]

rcuts = [get_cutoff(l, number_of_neighbors_in_rcut) for l in lattices]

# Create a matplotlib image
fig, ax = plt.subplots(len(noise_levels), len(lattices), constrained_layout=True, dpi=200)

for i, noise_level in enumerate(noise_levels):
    for j, lattice in enumerate(lattices):

        # calculate cutoff
        rcut = rcuts[j]

        # create a noisy structure
        noisy_struct = noisy_structure(lattice, noise_level)
        
        # compute descriptor
        image = noisy_struct.get_neighbor_map(0, r_cut=rcut)

        # display the image
        ax[i, j].imshow(image)
        
        # deactivate x and y scales as we show images
        ax[i, j].axis("off")
        ax[i, j].set_title(f"{bravais_lattices[j][1]}\n" + r"$\sigma =$" + f"{noise_level:.2f}", fontsize=4)

## 5. Getting ready for classification

- Are the images above different enough to be classified ?
- To answer, we will try to find a low-dimensional representation of the images that allows to discriminate them.
- We will use the PCA and LDA methods.
- We will try different cut-off radii, with different levels of noise, on different structures, so this will take approximately a minute (code is not optimized here)

In [ ]:
def create_atomic_structures(noise):

    # Lists to store data for the analysis
    structure_name  = []        # Store the names of the lattice types
    structure_label = []        # Store numerical labels for the lattice types (useful for coloring)
    all_structures  = []        # Store the flattened neighbor map images

    # Loop through each lattice type and its corresponding info (index and name)
    for i, (lattice, lat_num, lat_name) in enumerate(zip(lattices, lattice_num, lattice_name)):
        # Add noise to the lattice (standard deviation of 0.15 Angstroms)
        structure = noisy_structure(lattice, noise) 
        
        all_structures.append(structure)
        structure_name.append(lat_name)
        structure_label.append(lat_num)
    return all_structures, structure_label, structure_name

def compute_descriptors(structures, N_neigh = 64, img_size = 32, beta=3, gamma=2, at_per_struct=150, **kwargs):
    
    # Lists to store data for the analysis
    lattice_name_ = []      # Store the names of the lattice types
    lattice_num_ = []       # Store numerical labels for the lattice types (useful for coloring)
    all_images = []         # Store the flattened neighbor map images
    
    rng = np.random.default_rng(42) 
    
    # Loop through each lattice type and its corresponding info (index and name)
    for i, (lattice, lat_num, lat_name) in enumerate(zip(lattices, lattice_num, lattice_name)):
    
        # Add noise to the lattice (standard deviation of 0.15 Angstroms)
        structure = structures[i]
        
        cut_off = get_cutoff(lattice, N_neigh) # Calculate cutoff radius to include N_neigh neighbors
        
        structure.neighbor_map.beta=beta
        structure.neighbor_map.gamma=gamma
        
        # Choose n atoms randomly from the lattice
        for at in rng.choice(len(lattice), at_per_struct):  
        
            # Get the neighbor map for the selected atom and flatten it into a 1D array
            image = structure.get_neighbor_map(
                at,                                        # Index of the atom to focus on
                r_cut = cut_off,     
                img_target_size = img_size                 # Image is img_sizeximg_size pixels
            ).flatten()  # Convert 2D image to 1D for easier analysis
    
            # Store the image, lattice number, and lattice name
            all_images.append(image) 
            lattice_num_.append(lat_num)   
            lattice_name_.append(lat_name)
    
    # Convert the list of images to a NumPy array for further analysis (e.g., PCA, LDA)
    all_images = np.array(all_images) 
    return all_images, lattice_name_, lattice_num_

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib as mpl

def plot_pca(all_images, lattice_num_, noise_level, axes, column):
    """
    Plots PCA and LDA results for a given set of images and lattice numbers.

    Args:
        all_images: Array of flattened neighbor map images.
        lattice_num_: Array of numerical labels for the lattices.
        noise_level: Current noise level.
        axes: Axes objects for the subplots.
    """
    model1 = PCA(n_components=2)
    model2 = LinearDiscriminantAnalysis(n_components=2)

    for model_i, m in enumerate([model1, model2]):
        if model_i == 0:
            # PCA takes only X as input (unsupervised)
            model_result = m.fit_transform(all_images)
        else:
            # LDA takes X and Y as input (supervised)
            model_result = m.fit_transform(all_images, lattice_num_)

        axes[model_i, column].scatter(
            model_result[:, 0],
            model_result[:, 1],
            color=mpl.cm.tab10(lattice_num_),
            s=2,
            alpha=0.8
        )

        # a trick to add a legend 
        if (model_i==1) and (column==0):
            for i in range(len(lattices)):
                axes[1, 0].scatter([], [], s=2, color=mpl.cm.tab10(i), label=lattice_name[i])
            axes[model_i, column].legend(fontsize=5, frameon=False)
            
        axes[model_i, column].set_title(
            f"{['PCA', 'LDA'][model_i]}, " + r"$\sigma=$" + f"{noise_level:.2f}"
        )


def main_plot(noise_levels, number_of_neighbors_in_rcut, beta=3, gamma=2, img_size=32):
    fig, axes = plt.subplots(2, len(noise_levels), 
                            figsize=(10, 4), 
                            dpi=200, ## To adjust plot size 
                            constrained_layout=True)

    for i_noise, noise_level in enumerate(tqdm(noise_levels)):
        all_structures, structure_label, structure_name = create_atomic_structures(noise_level)
        all_images, lattice_name_, lattice_num_ = compute_descriptors(all_structures, 
                                                                    N_neigh=number_of_neighbors_in_rcut, 
                                                                    img_size=img_size, 
                                                                    beta=beta, 
                                                                    gamma=gamma,
                                                                    at_per_structure=300,
                                                                    )
        plot_pca(all_images, lattice_num_, noise_level, axes, i_noise)
        
    plt.savefig(f"pca_{number_of_neighbors_in_rcut}_beta{beta}_gamma{gamma}.pdf")
    plt.show()


In [ ]:
noise_levels = [0.1, 0.15, 0.2, 0.25, 0.3]

# Execution for different neighbor counts
for n_neigh in [32, 64, 128]:

    print(f"---------- {n_neigh} Neighbors ---------")
    main_plot(noise_levels, n_neigh, img_size=32)
    plt.show()

- Which method seems the most efficient for separating clusters ? Does it always work ?
- What is the unit or meaning of the plots axis ? 
- How could we improve the results ?
- Can you make the separation better for high noise conditions ?
- Is the uncorrelated noise used here representative of thermal noise ?
- Do all clusters have approx. the same variance ?
- What is the effect of the cut-off radius ?

## 6. Clustering methods

For the following, let's pick the condition "64 Neighbors / noise_std = 0.25A", where it can already be difficult to recognize structures.

Now, we will try to cluster the data in an unsupervised or supervised way.

To first get a sense of how noisy the structures are, let's visualize them.

In [ ]:
from ase.visualize import view
structure = noisy_structure(lattices[2], 0.25)

# Let's visualize the atoms coordinates to feel just how displaced atoms are for this level of noise

# Next line might work well or freeze the browser for a few seconds (or forever)
#view(structure, viewer='ngl')

In [ ]:
# A safer alternative visualization method (requires plotly): 

import plotly.graph_objects as go
import matplotlib as mpl


# Create the figure
fig = go.Figure()
fig.add_trace(
go.Scatter3d(
    x=structure.positions[:, 0],
    y=structure.positions[:, 1],
    z=structure.positions[:, 2],
    mode="markers",
    marker=dict(
        size=10,
        opacity=0.7,
    ),
)
)


# Set axis labels and title
fig.update_layout(
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="z",
    ),
    title="3D positions of the atoms",
    height=600,
    width=800
)
fig.layout.scene.camera.projection.type = "orthographic"

# Show the plot
fig.show()



In [ ]:
# here is the associated descriptor
# Play with values of gamma and beta to get the most out of the image

s = noisy_structure(lattices[2], 0.25)
s.neighbor_map.gamma=2/3
s.neighbor_map.beta=4
plt.imshow(s.get_neighbor_map(24, 8))
plt.axis("off")

**Dataset generation**

In [ ]:
# set a noise level
all_structures, structure_label, structure_name = create_atomic_structures(noise = 0.25)

# choose descriptor parameters
all_images, _, ground_truth_labels = compute_descriptors(all_structures,
                                                    N_neigh = 128,
                                                    img_size = 32, 
                                                    beta=3, 
                                                    gamma=2, 
                                                    at_per_struct=300)

**Unsupervised methods**

Gaussian Mixtures

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import adjusted_rand_score
    
#%matplotlib inline


def reduce_dimension(data, model="pca", perplexity=20, n_components=2):

    if model == "pca":
        model_lowd = PCA(n_components=n_components)
        low_dimension_imgs = model_lowd.fit_transform(all_images)

    elif model=="tsne":
        # Try this ? TSNE is a nonlinear manifold learning method
        # See https://distill.pub/2016/misread-tsne/
        low_dimension_imgs = TSNE(n_components=n_components, 
                                   init='random', 
                                   early_exaggeration=12,
                                   n_iter=1000, # called max_iter in recent versions >1.5 
                                   perplexity=perplexity # try 1, 20, 50, 100, 500
                                 ).fit_transform(all_images)
    else: 
        raise(ValueError("Unknown model"))
    return low_dimension_imgs

def clusterize_gm(data, ground_truth, ncomponents, n_init=100):

    gm = GaussianMixture(n_components=ncomponents, 
                         n_init=n_init, # Try 1, 10, 100... 
                         covariance_type="full", 
                         random_state=0, # for reproducible runs
                         #init_params="random_from_data" # "kmeans"
                        )
    
    gm.fit(data)

    clust = gm.predict(data)
    
    # A metric of unsupervised clustering methods performance is the Adjusted Rand Index 
    # (1: perfect agreement with ground truth, 0: equivalent to random assignment)
    score = adjusted_rand_score(ground_truth, clust)

    return clust, score

In [ ]:
# try model="tsne" and "pca"; perplexity is a parameter for tsne (not used by pca).
low_d_images = reduce_dimension(all_images, model="tsne", perplexity=40) 

# try different values of n_init, the number of attempts of the GM model.
clusters, score = clusterize_gm(low_d_images, ground_truth_labels, len(lattices), n_init=100)

# For fun, try to fit the images directly, without dimensionality reduction (will take a long time for n_init >> 1). Do you think the result is worth the wait ? 
# clusters, score = clusterize_gm(all_images, len(lattices))

print("The obtained score of similarity between the unsupervised clustering and ground truth labels:")
print("ARI: ", score)

fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=144, constrained_layout=True)
axes[0].set_title("Colored according to our unsupervised clustering")
axes[1].set_title("Colored according to ground truth labels")
axes[0].scatter(low_d_images[:, 0], 
            low_d_images[:, 1], 
            color=mpl.cm.tab10(clusters), 
            s=2, alpha=0.8)
axes[1].scatter(low_d_images[:, 0], 
            low_d_images[:, 1], 
            color=mpl.cm.tab10(ground_truth_labels), 
            s=2, alpha=0.8)

for i in np.unique(ground_truth_labels):
    axes[1].scatter([], [], s=2, color=mpl.cm.tab10(i), label=lattice_name[i])
axes[1].legend(fontsize=5, frameon=False)

- How does the method perform when the perplexity parameter varies ?
- Can you improve the clustering by tuning beta and gamma ?
- Can we do just as good with other unsupervised clustering methods such as KMeans or DBSCAN ?

**More unsupervised clustering methods**

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

# Dimensionality Reduction (Optional)
# you can try without this step
low_dimension_imgs = reduce_dimension(all_images, model="tsne") 

# K-means Clustering
kmeans = KMeans(n_clusters=len(lattices), random_state=0, n_init=100)  # Assuming 7 clusters
data_classes_kmeans = kmeans.fit_predict(low_dimension_imgs)

# DBSCAN Clustering
dbscan = DBSCAN(eps=10, min_samples=5)  # You'll need to tune eps and min_samples
data_classes_dbscan = dbscan.fit_predict(low_dimension_imgs)



# Visualization on the two first principal components
fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=144, constrained_layout=True)

# K-means
axes[0].scatter(low_dimension_imgs[:, 0], low_dimension_imgs[:, 1],
                c=mpl.cm.tab10(data_classes_kmeans), s=2, alpha=0.8)
axes[0].set_title("K-means Clustering")

# DBSCAN
axes[1].scatter(low_dimension_imgs[:, 0], low_dimension_imgs[:, 1],
                c=mpl.cm.tab10(data_classes_dbscan), s=2, alpha=0.8)
axes[1].set_title("DBSCAN Clustering")

plt.show()

# Evaluation (Adjusted Rand Index)
ari_kmeans = adjusted_rand_score(ground_truth_labels, data_classes_kmeans)
ari_dbscan = adjusted_rand_score(ground_truth_labels, data_classes_dbscan)

print("ARI (K-means):", ari_kmeans)
print("ARI (DBSCAN):", ari_dbscan)

- Try different values for `DBSCAN(eps=0.2, min_samples=10)`
- Is K-means a deterministic method ? Change the `random_state` variable

## Defect detection

Here we try to identify the environment of a vacancy, again in an unsupervised way

In [ ]:
import ase.build
from neighbors_map import Atoms

In [ ]:

# You can set the element to "Al" for fcc, Po for simple cubic, Si for diamond etc
bulk_Fe = ase.build.bulk("Fe", cubic=True) * [6, 6, 6]
# Level of noise, can be modified 
bulk_Fe.rattle(0.08)

# Create a copy of the system where an atom is missing. The atom is at the center of the cell.
vacancy = bulk_Fe.copy()
id_to_del = np.argmin(np.linalg.norm(bulk_Fe.positions - bulk_Fe.get_cell()[0, 0]//2, 
                                    axis=1)
                            )

del vacancy[id_to_del]

bulk_Fe = Atoms(bulk_Fe)
vacancy = Atoms(vacancy)

# Set the neighbor maps parameters 
vacancy.neighbor_map.beta=3
vacancy.neighbor_map.gamma=0.5

In [ ]:
# Accumulate the descriptors of the system with vacancy in an array
descriptors_bulk = []

for i, _ in enumerate(bulk_Fe):
    nm = bulk_Fe.get_neighbor_map(i, 8.0)
    descriptors_bulk.append(nm.flatten())
    
descriptors_bulk = np.array(descriptors_bulk)

# Accumulate the descriptors of the system with vacancy in an array
descriptors_vac = []

for i, _ in enumerate(vacancy):
    nm = vacancy.get_neighbor_map(i, 8.0)
    descriptors_vac.append(nm.flatten())
    
descriptors_vac = np.array(descriptors_vac)

In [ ]:
# Reduce dimension with PCA & project descriptors
pca = PCA(n_components=2)
projected_desc_vac = pca.fit_transform(descriptors_vac)

# TSNE performed poorly in my tests, but maybe you can do better ? 
# projected_desc_vac = TSNE(n_components=2, 
#                                    init='random', 
#                                    early_exaggeration=12,
#                                    n_iter=1000, # called max_iter in recent versions >1.5 
#                                    perplexity=40 # try 1, 20, 50, 100, 500
#                                  ).fit_transform(descriptors_vac)

In [ ]:
import sklearn.mixture

# Apply 2-class GMM
gmm = sklearn.mixture.GaussianMixture(n_components=2, n_init=20, )

clusters_vacancy = gmm.fit_predict(projected_desc_vac)

# Plot the PCA and the performed clustering 
plt.scatter(projected_desc_vac[:,0], projected_desc_vac[:,1], color=[mpl.cm.tab10(cv) for cv in clusters_vacancy])

# Put the atoms' cartesian positions in a list, grouped per-cluster

clusters_cartesian_positions = [vacancy.positions[clusters_vacancy==i] for i in np.unique(clusters_vacancy)]

In [ ]:

import plotly.graph_objects as go
import matplotlib as mpl


# Plot the defective atoms

# Create the figure
fig = go.Figure()

for c_i, c in enumerate(sorted(clusters_cartesian_positions, key=len)):
    
    level = np.logspace(0, -2, len(clusters_cartesian_positions))[c_i]
    fig.add_trace(
    go.Scatter3d(
        x=c[:, 0],
        y=c[:, 1],
        z=c[:, 2],
        mode="markers",
        marker=dict(
            size=10,
            opacity=level,
            color=level
        ),
    )
    )

# Set axis labels and title
fig.update_layout(
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="z",
    ),
    title="3D positions of the atoms",
    height=600,
    width=800
)
fig.layout.scene.camera.projection.type = "orthographic"

# Show the plot
fig.show()

